In [ ]:
#Importing Essential Libraries and Modules

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.tensorboard import SummaryWriter
import timm
from torchvision.transforms import CenterCrop

In [ ]:
torch.manual_seed(0)


class MaxCenterCrop:
    def __call__(self, sample):
        min_size = min(sample.size[0], sample.size[1])
        return CenterCrop(min_size)(sample)


In [ ]:
#Setting up the batch size and device to GPU 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size = 16



In [ ]:
#Creating data loaders for training  and validation.

data_dir = 'data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/val'

data_transforms_train = transforms.Compose([MaxCenterCrop(),
                                            transforms.Resize(256),
                                            transforms.ToTensor()])

data_transforms_validation = transforms.Compose([MaxCenterCrop(),
                                            transforms.Resize(256),
                                            transforms.ToTensor()])

image_dataset_train = datasets.ImageFolder(train_dir, transform = data_transforms_train)

image_dataset_validation = datasets.ImageFolder(valid_dir, transform = data_transforms_validation)


dataloader_train = torch.utils.data.DataLoader(image_dataset_train, batch_size=batch_size,
                                                shuffle=True)

dataloader_valid = torch.utils.data.DataLoader(image_dataset_validation, batch_size=batch_size,
                                                shuffle=True)



In [ ]:
#Uploading the efficientnet_b7 pretrained model and modification


class_names = image_dataset_train.classes

model = timm.create_model('efficientnet_b7', pretrained=True)

num_ftrs = model.classifier.in_features

model.classifier = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)


In [ ]:
#Setting up the optimizer and the criterion

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()



In [ ]:
from ignite.engine import Engine, _prepare_batch, create_supervised_trainer


trainer = create_supervised_trainer(model, optimizer, criterion, device)

In [ ]:
#Metrics

from ignite.metrics import Loss, TopKCategoricalAccuracy, Precision, Recall


metrics = {
    'loss': Loss(criterion),
    'TopKCategoricalAccuracy1': TopKCategoricalAccuracy(k=1),
    'TopKCategoricalAccuracy5': TopKCategoricalAccuracy(k=5),
    'avg_precision': Precision(average=True), 
    'avg_recall': Recall(average=True)
}



In [ ]:
#Progress bar

from ignite.contrib.handlers.tqdm_logger import ProgressBar

pbar = ProgressBar(bar_format='')
pbar.attach(trainer, output_transform=lambda x: {'loss': x})



In [ ]:
#Defining evaluators

from ignite.engine import create_supervised_evaluator

train_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)
val_evaluator = create_supervised_evaluator(model, metrics=metrics, device=device)



In [ ]:
from torch.utils.data.dataset import Subset

random_indices = np.random.permutation(np.arange(len(image_dataset_train)))[:len(image_dataset_validation)]

train_subset = Subset(image_dataset_train, indices=random_indices)

train_eval_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True, 
                               drop_last=True, pin_memory="cuda")

In [ ]:
#Metrics computation

from ignite.engine import Events

@trainer.on(Events.EPOCH_COMPLETED)
def compute_and_display_offline_train_metrics(engine):
    epoch = engine.state.epoch
    print("Compute train metrics...")
    metrics = train_evaluator.run(train_eval_loader).metrics
    print("Training Results - Epoch: {}  Loss: {:.4f} | TopKCategoricalAccuracy1: {:.4f} | Precision: {:.4f} | Recall: {:.4f}"
          .format(engine.state.epoch, metrics['loss'], metrics['TopKCategoricalAccuracy1'], metrics['avg_precision'], metrics['avg_recall']))
    
    
@trainer.on(Events.EPOCH_COMPLETED)
def compute_and_display_val_metrics(engine):
    epoch = engine.state.epoch
    print("Compute validation metrics...")
    metrics = val_evaluator.run(dataloader_valid).metrics
    print("Validation Results - Epoch: {}  Loss: {:.4f} | TopKCategoricalAccuracy1: {:.4f} | Precision: {:.4f} | Recall: {:.4f}"
          .format(engine.state.epoch, metrics['loss'], metrics['TopKCategoricalAccuracy1'], metrics['avg_precision'], metrics['avg_recall']))  



In [ ]:
#Run training


Start_one = time.time()

max_epochs = 3

output = trainer.run(dataloader_train, max_epochs=max_epochs)

End_one = Start_one - time.time()

print('Training complete in {:.0f}m {:.0f}s'.format(End_one // 60, End_one % 60))